In [2]:
import requests

import pandas as pd
import pandasql as ps
pd.options.display.float_format = '{:,}'.format

In [3]:
filename = '../data/exchange_metrics.csv'
df_exchanges = pd.read_csv(filename)

In [4]:
df_exchanges.iloc[0]

coin_id                                                                    cosmos
coin_name                                                                  Cosmos
symbol                                                                       atom
base                            IBC/27394FB092D2ECCD56123C74F36E4C1F926001CEAD...
target                                                                      UOSMO
market_name                                                               Osmosis
market_identifier                                                         osmosis
market_has_trading_incentive                                                False
price                                                                 4.714796485
volume                                                               32,373,181.0
pp_total_volume                                                0.0130332713490066
is_stale                                                                    False
Name: 0, dtype: 

In [23]:
filename = '../data/coin_metrics.csv'
df_coins = pd.read_csv(filename)

In [5]:
# df_coins.iloc[0]

### Exchange Metrics

In [25]:
sql_string = """
    WITH num_exchanges AS (
        SELECT coin_name, symbol, COUNT(DISTINCT market_name) AS num_exchanges
        FROM df_exchanges
        GROUP BY 1, 2
    ),
    
    top_exchange_by_volume AS (
        SELECT A.coin_name, A.symbol, A.market_name, B.max_volume
        FROM df_exchanges AS A
        INNER JOIN (
            SELECT coin_name, symbol, MAX(volume) AS max_volume
            FROM df_exchanges
            GROUP BY 1, 2
        ) AS B
        ON 
            A.coin_name = B.coin_name AND
            A.symbol = B.symbol AND 
            A.volume = B.max_volume
        GROUP by 1, 2, 3
    )    
    
    SELECT 
        DISTINCT
        A.coin_name, 
        A.symbol, 
        A.price, 
        B.num_exchanges,
        C.market_name,
        max_volume AS market_volume,
        volume_24h,
        ROUND(max_volume*1./ volume_24h*1., 2) AS pp_volume
    FROM df_coins AS A 
    JOIN num_exchanges AS B
    ON 
        A.coin_name = B.coin_name AND 
        A.symbol = B.symbol 
    JOIN top_exchange_by_volume AS C
    ON 
        A.coin_name = C.coin_name AND 
        A.symbol = C.symbol
    ORDER BY volume_24h DESC
"""

df_final = ps.sqldf(sql_string)

In [26]:
df_final.head(20)

,coin_name,symbol,price,num_exchanges,market_name,market_volume,volume_24h,pp_volume
0,Cosmos,atom,39.13,46,Binance,"410,390,194.0","2,483,887,593.0",0.17
1,Polkadot,dot,26.57,52,Binance,"453,084,111.0","1,778,582,619.0",0.25
2,Avalanche,AVAX,95.27,54,Binance,"268,568,234.0","1,300,748,454.0",0.21
3,Crypto.com Coin,cro,0.507124,43,Crypto.com Exchange,"74,447,910.0","351,203,372.0",0.21
4,Band Protocol,band,5.8,57,Binance,"37,955,079.0","136,099,485.0",0.28
5,Fetch.ai,fet,0.544105,29,Binance,"53,102,437.0","125,737,395.0",0.42
6,Osmosis,osmo,8.33,3,Osmosis,"32,373,181.0","83,287,642.0",0.39
7,Kava,kava,4.22,30,Binance,"38,280,414.0","81,986,583.0",0.47
8,Quant,qnt,166.36,30,Coinbase Exchange,"12,581,331.0","71,581,040.0",0.18
9,ICON,icx,1.13,33,Upbit,"16,773,106.0","61,118,427.0",0.27
